In [14]:
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import keras
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [2]:
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_8272392662f948bfb63722433ce82e01.csv"
df=pd.read_csv(url)
df.head()

,ID,Age,Years_at_Employer,Years_at_Address,Income,Credit_Card_Debt,Automobile_Debt,Outcomes
0,1,32.527993,9.388112,0.297587,37843.67902,-3246.656516,-4794.706076,0
1,2,34.578226,11.972864,1.485103,65765.22990,-15597.775660,-17632.168590,1
2,3,37.697188,12.459833,0.085444,61002.28547,-11401.917620,-7910.242809,1
3,4,28.684510,1.387144,1.837598,19952.73494,-1233.378450,-2408.097355,0
4,5,32.614672,7.488821,0.234122,24970.12812,-1135.680543,-397.323189,0


In [3]:
df.shape

(400, 8)

In [4]:
df.groupby("Outcomes").count().T

Outcomes,0,1
ID,300,100
Age,300,100
Years_at_Employer,300,100
Years_at_Address,300,100
Income,300,100
Credit_Card_Debt,300,100
Automobile_Debt,300,100


In [5]:
df_1=df[df["Outcomes"]==1]
df_1=pd.concat([df_1]*3,axis=0)
df_1.shape

(300, 8)

In [6]:
df_0=df[df["Outcomes"]==0]
df=pd.concat([df_1,df_0],axis=0).sample(frac=1)
df.head()

,ID,Age,Years_at_Employer,Years_at_Address,Income,Credit_Card_Debt,Automobile_Debt,Outcomes
132,133,33.713802,12.704102,0.520628,24672.21349,-2293.944643,-7103.669764,0
254,255,34.993071,0.691826,0.706480,31936.80051,-2755.826680,-4145.885974,1
26,27,39.681779,4.287113,1.743599,73552.82708,-1711.380119,-1285.282215,0
237,238,34.444818,9.430418,0.738051,42971.20868,-1770.804456,-8394.316940,0
366,367,30.881326,0.262054,0.165898,22259.73726,-808.793103,137.640816,1


In [7]:
df.shape

(600, 8)

In [8]:
X=df.drop(["ID","Outcomes"],axis=1)
X.head()

,Age,Years_at_Employer,Years_at_Address,Income,Credit_Card_Debt,Automobile_Debt
132,33.713802,12.704102,0.520628,24672.21349,-2293.944643,-7103.669764
254,34.993071,0.691826,0.706480,31936.80051,-2755.826680,-4145.885974
26,39.681779,4.287113,1.743599,73552.82708,-1711.380119,-1285.282215
237,34.444818,9.430418,0.738051,42971.20868,-1770.804456,-8394.316940
366,30.881326,0.262054,0.165898,22259.73726,-808.793103,137.640816


In [9]:
y=df[["Outcomes"]]
y.head()

,Outcomes
132,0
254,1
26,0
237,0
366,1


In [10]:
scaler=preprocessing.StandardScaler()
names=X.columns
X=scaler.fit_transform(X)
X=pd.DataFrame(X,columns=names)
X.head()

,Age,Years_at_Employer,Years_at_Address,Income,Credit_Card_Debt,Automobile_Debt
0,-0.015568,0.749229,-0.473103,-0.472218,0.287038,-0.010131
1,0.139675,-1.054607,-0.164757,-0.317945,0.193921,0.343647
2,0.708665,-0.514717,1.555915,0.565826,0.404485,0.685801
3,0.073143,0.257632,-0.112378,-0.083615,0.392505,-0.164504
4,-0.359298,-1.119145,-1.061630,-0.523450,0.586451,0.855996


In [11]:
ienc=OneHotEncoder(handle_unknown="ignore")
y=ienc.fit_transform(y)
y=pd.DataFrame(y.toarray())
y.head()

,0,1
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(480, 6)
(120, 6)


In [13]:
length=X_train.shape[1]
num_classes=y_test.shape[1]
print(length)
print(num_classes)

6
2


In [25]:
def classification_model():
    # create model
    model = Sequential()
    model.add(Dense(length, activation='relu', input_shape=(length,)))
    model.add(Dense(200, activation='relu'))
    model.add(Dense(200,activation="relu"))
    model.add(Dense(num_classes, activation='softmax'))
    opt=Adam(lr=0.008)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [69]:
model=classification_model()
model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=260, verbose=2)
model.evaluate(X_test,y_test,verbose=0)

Epoch 1/260
15/15 - 0s - loss: 0.6411 - accuracy: 0.6208 - val_loss: 0.5195 - val_accuracy: 0.7083
Epoch 2/260
15/15 - 0s - loss: 0.5334 - accuracy: 0.7125 - val_loss: 0.5140 - val_accuracy: 0.6917
Epoch 3/260
15/15 - 0s - loss: 0.5198 - accuracy: 0.7396 - val_loss: 0.4903 - val_accuracy: 0.7917
Epoch 4/260
15/15 - 0s - loss: 0.5038 - accuracy: 0.7437 - val_loss: 0.4930 - val_accuracy: 0.7000
Epoch 5/260
15/15 - 0s - loss: 0.4883 - accuracy: 0.7771 - val_loss: 0.4678 - val_accuracy: 0.7667
Epoch 6/260
15/15 - 0s - loss: 0.4734 - accuracy: 0.7688 - val_loss: 0.4716 - val_accuracy: 0.7833
Epoch 7/260
15/15 - 0s - loss: 0.4817 - accuracy: 0.7708 - val_loss: 0.4674 - val_accuracy: 0.7667
Epoch 8/260
15/15 - 0s - loss: 0.4671 - accuracy: 0.7750 - val_loss: 0.4655 - val_accuracy: 0.7833
Epoch 9/260
15/15 - 0s - loss: 0.4515 - accuracy: 0.7896 - val_loss: 0.4636 - val_accuracy: 0.7833
Epoch 10/260
15/15 - 0s - loss: 0.4580 - accuracy: 0.7833 - val_loss: 0.4943 - val_accuracy: 0.7417
Epoch 11/

15/15 - 0s - loss: 0.2756 - accuracy: 0.8771 - val_loss: 0.3392 - val_accuracy: 0.8417
Epoch 84/260
15/15 - 0s - loss: 0.2332 - accuracy: 0.8917 - val_loss: 0.4534 - val_accuracy: 0.8750
Epoch 85/260
15/15 - 0s - loss: 0.1943 - accuracy: 0.9250 - val_loss: 0.2949 - val_accuracy: 0.8583
Epoch 86/260
15/15 - 0s - loss: 0.1082 - accuracy: 0.9417 - val_loss: 0.2157 - val_accuracy: 0.9083
Epoch 87/260
15/15 - 0s - loss: 0.0956 - accuracy: 0.9542 - val_loss: 0.2281 - val_accuracy: 0.9250
Epoch 88/260
15/15 - 0s - loss: 0.0825 - accuracy: 0.9708 - val_loss: 0.1933 - val_accuracy: 0.9417
Epoch 89/260
15/15 - 0s - loss: 0.0736 - accuracy: 0.9792 - val_loss: 0.2619 - val_accuracy: 0.9250
Epoch 90/260
15/15 - 0s - loss: 0.0738 - accuracy: 0.9750 - val_loss: 0.1797 - val_accuracy: 0.9500
Epoch 91/260
15/15 - 0s - loss: 0.0794 - accuracy: 0.9792 - val_loss: 0.2162 - val_accuracy: 0.9250
Epoch 92/260
15/15 - 0s - loss: 0.1089 - accuracy: 0.9583 - val_loss: 0.2950 - val_accuracy: 0.9250
Epoch 93/260


Epoch 165/260
15/15 - 0s - loss: 0.0431 - accuracy: 0.9896 - val_loss: 0.5402 - val_accuracy: 0.9083
Epoch 166/260
15/15 - 0s - loss: 0.0472 - accuracy: 0.9812 - val_loss: 0.5184 - val_accuracy: 0.9083
Epoch 167/260
15/15 - 0s - loss: 0.0505 - accuracy: 0.9812 - val_loss: 0.5365 - val_accuracy: 0.9167
Epoch 168/260
15/15 - 0s - loss: 0.0488 - accuracy: 0.9833 - val_loss: 0.5095 - val_accuracy: 0.9250
Epoch 169/260
15/15 - 0s - loss: 0.0472 - accuracy: 0.9792 - val_loss: 0.4582 - val_accuracy: 0.9000
Epoch 170/260
15/15 - 0s - loss: 0.1597 - accuracy: 0.9583 - val_loss: 0.4333 - val_accuracy: 0.9000
Epoch 171/260
15/15 - 0s - loss: 0.1124 - accuracy: 0.9500 - val_loss: 0.4959 - val_accuracy: 0.9000
Epoch 172/260
15/15 - 0s - loss: 0.1317 - accuracy: 0.9625 - val_loss: 0.2693 - val_accuracy: 0.9250
Epoch 173/260
15/15 - 0s - loss: 0.0973 - accuracy: 0.9625 - val_loss: 0.2752 - val_accuracy: 0.9250
Epoch 174/260
15/15 - 0s - loss: 0.2092 - accuracy: 0.9479 - val_loss: 0.2495 - val_accurac

15/15 - 0s - loss: 0.0168 - accuracy: 0.9937 - val_loss: 0.4089 - val_accuracy: 0.9167
Epoch 247/260
15/15 - 0s - loss: 0.0177 - accuracy: 0.9958 - val_loss: 0.4204 - val_accuracy: 0.9167
Epoch 248/260
15/15 - 0s - loss: 0.0269 - accuracy: 0.9917 - val_loss: 0.3166 - val_accuracy: 0.9250
Epoch 249/260
15/15 - 0s - loss: 0.0240 - accuracy: 0.9917 - val_loss: 0.4600 - val_accuracy: 0.9250
Epoch 250/260
15/15 - 0s - loss: 0.0244 - accuracy: 0.9854 - val_loss: 0.2799 - val_accuracy: 0.9417
Epoch 251/260
15/15 - 0s - loss: 0.0230 - accuracy: 0.9937 - val_loss: 0.3990 - val_accuracy: 0.9250
Epoch 252/260
15/15 - 0s - loss: 0.0352 - accuracy: 0.9875 - val_loss: 0.4838 - val_accuracy: 0.9000
Epoch 253/260
15/15 - 0s - loss: 0.0339 - accuracy: 0.9875 - val_loss: 0.3694 - val_accuracy: 0.8917
Epoch 254/260
15/15 - 0s - loss: 0.0575 - accuracy: 0.9812 - val_loss: 0.4061 - val_accuracy: 0.8833
Epoch 255/260
15/15 - 0s - loss: 0.0894 - accuracy: 0.9750 - val_loss: 0.6288 - val_accuracy: 0.8833
Epoc

[0.25135478377342224, 0.925000011920929]

In [70]:
yhat=model.predict(X_test)
print("AUC on test data: ", roc_auc_score(y_test,yhat))

AUC on test data:  0.9679754942912838
